<a href="https://colab.research.google.com/github/derekphilipau/machinelearningforartists/blob/main/stylegan2_ada_pytorch_resume_vases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning with Vases

In this notebook we will train our dataset using a pre-trained model of vases.

# SETUP (MUST RUN EACH TIME)

## Verify Runtime is GPU

In the menu, select Runtime -> Change Runtime Type and verify you are using the **GPU**.  Also select **High-RAM** if you are using Colab Pro.

The `nvidia-smi` command below should **NOT** display *"NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver."*

In [ ]:
!nvidia-smi -L

## Mount your Google Drive

You will be storing the training models and progress images on your Google Drive.  This is very convenient for viewing progress, and if your Colab notebook is disconnected you will not lose your models.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Install Stylegan2-ada-pytorch Prerequisites

In [3]:
import torch

Verify the next command results in "1".  If not, go back to the beginning and verify you have a GPU runtime.

In [ ]:
torch.cuda.device_count()

In [5]:
import torchvision

In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

## Get the StyleGAN code

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

In [8]:
mkdir /content/stylegan2-ada-pytorch/datasets

In [ ]:
%cd /content/stylegan2-ada-pytorch

# FIRST RUN (RUN ONLY ONCE)

## Create Folders on Your Google Drive

If we accidentally close our browser or the Colab runtime disconnects, we will lose all of our training models and progress images.  Therefore we want to store the training data on our Google Drive.  The following cells will create a new folder on your Google Drive, **MachineLearningForArtists**.  Your training data will be stored in **MachineLearningForArtists/MyProject1024**

In [ ]:
mkdir /content/drive/MyDrive/MachineLearningForArtists

In [11]:
mkdir /content/drive/MyDrive/MachineLearningForArtists/MyProject1024

If you get an error "cannot create directory", it's probably because the folder already exists on your Google Drive and you can ignore the error.

## Prepare Your Image Dataset for StyleGAN


### Unzip your dataset

***If your images are not zipped, skip this step.***

If your images are in a .zip file, you will need to unzip them.   Copy the path to your .zip file and paste here under `zip_path`. 


In [ ]:
zip_path = '' #@param {type:"string"}

In [ ]:
!unzip "$zip_path"

### Prepare your dataset for StyleGAN


Copy the path to your unzipped dataset folder of images and paste into `dataset_path`

In [12]:
dataset_path = '' #@param {type:"string"}

The following commands will prepare your dataset.  It will crop all of your images to 1024x1024 pixels.  ***It will NOT overwrite your original images.***

In [ ]:
!apt install imagemagick webp

In [14]:
!mkdir /content/stylegan2-ada-pytorch/datasets/raw

In [16]:
!mkdir /content/stylegan2-ada-pytorch/datasets/processed

In [15]:
!cp "$dataset_path"/* /content/stylegan2-ada-pytorch/datasets/raw/

In [ ]:
%cd /content/stylegan2-ada-pytorch/datasets/raw/

In [18]:
!mogrify -type TrueColor -set colorspace sRGB -colorspace sRGB -resize 1024x1024 -background white -gravity center -extent 1024x1024 -format jpg -path ../processed *

In [ ]:
cd /content/stylegan2-ada-pytorch/

In [ ]:
!python dataset_tool.py --source=/content/stylegan2-ada-pytorch/datasets/processed/ --dest=./datasets/stylegan_dataset1024.zip

In [22]:
!cp ./datasets/stylegan_dataset1024.zip /content/drive/MyDrive/MachineLearningForArtists/MyProject1024/

## Train with Transfer Learning using the Vases model

This cell block will begin to train with your dataset using an existing .pkl model that has been trained with vases.

Download the vases model:

In [ ]:
!gdown --id 1-4gsVVtmD3hGapd2XJr-ovzYSOARHsJ5

In [ ]:
!python train.py --resume=/content/stylegan2-ada-pytorch/stylegan2-ada-pytorch-vases-9712.pkl --outdir=/content/drive/MyDrive/MachineLearningForArtists/MyProject1024 --data=./datasets/stylegan_dataset1024.zip --gpus=1 --cfg=paper1024 --mirror=1 --snap=10 --metrics=none

## Go to bed :)

Normal training of a dataset can take days.  Transfer learning will be quicker.

Results are stored in your Google Drive under drive/MyDrive/MachineLearningForArtists/MyProject

Each training run is stored in a separate directory.  The initial training run is stored in `00000-stylegan_dataset...`.  If you run training twice, the second run will be stored in `00001-stylegan_dataset...`.  And so on.

Inside the training run directory you will see various files.  `reals.png` shows a sample of the training dataset.  In it you should see various images from your original dataset. `fakes000000.png` is a sample of generated images from the initial model.  `network-snapshot-XXXXXX.pkl` is the actual model which can be used later to generate "fake" images, videos, etc.

As training progresses you will see more `fakes` & `network-snapshot` files created.


# RESUME TRAINING

***Make sure you run the above section "SETUP (MUST RUN EACH TIME)" before resuming training.***


In [ ]:
%cd /content/stylegan2-ada-pytorch

Copy previously saved datset

In [ ]:
!cp /content/drive/MyDrive/MachineLearningForArtists/MyProject1024/stylegan_dataset1024.zip ./datasets/

If you had to stop training, the browser window closed, or the Colab session timed out (12-24 hours), you can resume training by using a .pkl file stored on your Google Drive.  For your project, this file will be in `drive/MyDrive/MachineLearningForArtists/MyProject1024/0000X-stylegan_datset...`

In [ ]:
#@title PKL File Path 
#@markdown Copy & Paste .pkl file path here:
pkl_file = '' #@param {type:"string"}


Run the training, resuming with the last save .pkl file.  Results will be stored in your Google Drive under `MachineLearningForArtists/MyProject1024`

In [ ]:
cd /content/stylegan2-ada-pytorch/

In [ ]:
!python train.py --resume=$pkl_file --outdir=/content/drive/MyDrive/MachineLearningForArtists/MyProject1024 --data=./datasets/stylegan_dataset1024.zip --gpus=1 --cfg=paper1024 --mirror=1 --snap=10 --metrics=none
